In [1]:
using Revise
using TiSR
using DynamicQuantities
using Plots
using GraphPlot
using Compose
using Graphs
using LayeredLayouts
using SymbolicUtils
using Cairo
using Fontconfig

TiSR.jl loaded.

In [5]:
x = log10(3.5735e-6)
println(x)
y = 10^x
println(y)

-5.446906213562814
3.573500000000002e-6


In [9]:
# or with synthetic data # -------------------------------------------------------------------------
data_matr = rand(100, 5)
data_matr[:, end] .= 3.0 .* (data_matr[:, 1] .* 5.0 .+ data_matr[:, 2]) .^ 7.0 + exp.(data_matr[:, 1] .* 5.0 .+ data_matr[:, 2])
# data_matr[:, end] .= (data_matr[:, 1].^2 .+ data_matr[:, 2].^2).^0.5
# data_matr[:, end] .= sin.(data_matr[:, 1]) .+ cos.(data_matr[:, 2])
# -> 3 * (v1 * 5 + v2)^7 + exp(v1 * 5 + v2)

# prepare remainder for settings # -----------------------------------------------------------------
fit_weights = 1 ./ data_matr[:, end] # weights to minimize relative deviation
arbitrary_name = ""
parts = [0.8, 0.2]

pow_abs(v1, v2) = abs(v1)^v2
sqrt_abs(v1) = sqrt(abs(v1))
pow2(v1) = v1^2
pow3(v1) = v1^3

ops, data = Options(
    data_descript=data_descript(
        data_matr;
        arbitrary_name = arbitrary_name,
        parts          = parts,
        fit_weights    = fit_weights
    ),
    general=general_params(
        n_gens          = typemax(Int64),
        pop_size        = 500,
        max_compl       = 30,
        pow_abs_param   = true,
        prevent_doubles = 1e-2,
        t_lim           = 60.0 * 1.0,
        multithreadding = false,
        init_tree_depth = 6,
    ),
    fitting=fitting_params(
        early_stop_iter = 5,
        max_iter        = 15,
        lasso_factor    = 1e-7,
    ),
    p_unaops      = (1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,  1.0),
    unaops        = (exp, log, sin, cos, abs, sqrt_abs, pow2, pow3),
    p_binops      = (1.0, 1.0, 1.0, 1.0, 1.0),
    binops        = (+,   -,   *,   /,   ^  ),
    illegal_dict = Dict(:sin => (sin, cos),
                    :cos => (sin, cos),
                    :abs => (abs,),
                    :exp => (exp, log),
                    :log => (exp, log),),
     );



[80, 100]

┌ Warning: a high prevent_doubles may filter non-equal individuals
└ @ TiSR /Users/juliareu/GitCode/TiSR/src/options.jl:105
┌ Warning: ^ is only valid for positive bases. Otherwise provide a pow(abs(x), y) function with its own drawbacks.
└ @ TiSR /Users/juliareu/GitCode/TiSR/src/options.jl:39


In [3]:
struct Unit{Q<:AbstractQuantity}
    value::Q
    joker::Bool
    penalty::Int64
end

ustrip(w::Unit)     = ustrip(w.value)                                   # ustrip removes the unit from the quantity
dimension(w::Unit)  = DynamicQuantities.dimension(w.value)              # dimension returns the dimension of the quantity
valid(x::Unit)      = !(x.penalty > 0)                                  # valid returns true if no corrections had to be made up until the current nodes
penalty(x::Unit)    = x.penalty                                         # penalty returns the penalty of the unit
joker(x::Unit)      = x.joker                                           # joker returns true if the unit is a joker
value(x::Unit)      = x.value                                           # value returns the quantity

function dim_analysis!(node::TiSR.Node{T}, quantities::Vector{Q}, ops, correct::Bool; potential = false) where {T<:Number, Q<:AbstractQuantity{T}}
    #pot = false
    if node.ari == -1 #constant
        add_unit!(node, "[?], $(0)")
        return Unit{Q}(node.val * quantities[end], true, 0), false

    elseif node.ari == 0 #variable
        add_unit!(node, "[$(quantities[node.ind])], $(0)")
        return Unit{Q}((node.val * quantities[node.ind]), false, 0), false
        
    elseif node.ari == 1 #unary operation
        lef, pot = dim_analysis!(node.lef, quantities, ops, correct, potential = potential) #lef is of type Unit
        pot && potential && return Unit(quantities[end], false, 0), pot #returns dummy Unit and true to find all nodes which could potentially be corrected
        op = Symbol(ops.unaops[node.ind])

        if op in (:sin, :cos, :exp, :log, :log10)
            #check if lef is dimensionless
            if iszero(dimension(lef))
                add_unit!(node, "[0], $(penalty(lef))")
                return Unit(Quantity(quantities[end]), false, penalty(lef)), pot
            elseif potential
                return Unit(quantities[end], false, 0), true #returns dummy Unit and true to find all nodes which could potentially be corrected
            elseif correct #unit will be corrected through constant
                # correct unit of lef using a constant
                new_times = Node(2, findfirst(isequal(*), ops.binops))
                add_unit!(new_times, "[], $(penalty(lef)+1)")
                new_times.lef = Node(rand())
                add_unit!(new_times.lef, "[], $(penalty(lef))")
                new_times.rig = node.lef
                node.lef = new_times
                add_unit!(node, "[], $(penalty(lef)+1)")
                return Unit(quantities[end], false, penalty(lef) + 1), true
            else #unit will not be corrected, only penalty added and the current unit returned
                add_unit!(node, "[0], $(penalty(lef)+1)")
                return Unit(Quantity(quantities[end]), false, penalty(lef) + 1), true
            end
        elseif op in (:sqrt, :sqrt_abs)
            if joker(lef)
                add_unit!(node, "[?], $(penalty(lef))")
            else
                add_unit!(node, "[$(DynamicQuantities.dimension(sqrt(value(lef))))], $(penalty(lef))")
            end
            return Unit(sqrt(value(lef)), joker(lef), penalty(lef)), pot
        elseif op in (:abs,)
            if joker(lef)
                add_unit!(node, "[?], $(penalty(lef))")
            elseif iszero(dimension(lef))
                add_unit!(node, "[0], $(penalty(lef))")
            else
                add_unit!(node, "[$(DynamicQuantities.dimension(value(lef)))], $(penalty(lef))")
            end
            return Unit(abs(value(lef)), joker(lef), penalty(lef)), pot
        elseif op in (:pow2,)
            if joker(lef)
                add_unit!(node, "[?], $(penalty(lef))")
            else
                add_unit!(node, "[$(DynamicQuantities.dimension(value(lef))^2)], $(penalty(lef))")
            end
            return Unit(value(lef)^2, joker(lef), penalty(lef)), pot
        elseif op in (:pow3,)
            if joker(lef)
                add_unit!(node, "[?], $(penalty(lef))")
            else
                add_unit!(node, "[$(DynamicQuantities.dimension(value(lef))^3)], $(penalty(lef))")
            end
            return Unit(value(lef)^3, joker(lef), penalty(lef)), pot
        end

    elseif node.ari == 2
        lef, pot = dim_analysis!(node.lef, quantities, ops, correct, potential = potential)
        pot && potential && return Unit(quantities[end], false, 0), true #returns dummy Unit and true to find all nodes which could potentially be corrected
        rig, pot = dim_analysis!(node.rig, quantities, ops, correct, potential = potential)
        pot && potential && return Unit(quantities[end], false, 0), true #returns dummy Unit and true to find all nodes which could potentially be corrected
        
        op = Symbol(ops.binops[node.ind])
        if op in (:+, :-)
            if dimension(lef) == dimension(rig)
                if joker(lef) && joker(rig)
                    add_unit!(node, "[?], $(penalty(lef)+penalty(rig))")
                elseif joker(lef)
                    add_unit!(node, "[$(dimension(rig))], $(penalty(lef)+penalty(rig))")
                elseif joker(rig)
                    if iszero(dimension(lef))
                        add_unit!(node, "[0], $(penalty(lef)+penalty(rig))")
                    else
                        add_unit!(node, "[$(dimension(lef))], $(penalty(lef)+penalty(rig))")
                    end
                else
                    add_unit!(node, "[$(dimension(lef))], $(penalty(lef)+penalty(rig))")
                end
                return Unit(value(lef), joker(lef), penalty(lef)+penalty(rig)), pot

            elseif true in (joker(lef), joker(rig)) # no correction if either one is already a joker (would add unnecessary constants)
                add_unit!(node, joker(lef) == true ? "[$(dimension(rig))], $(penalty(lef)+penalty(rig))" : "[$(dimension(lef))], $(penalty(lef)+penalty(rig))")
                return joker(lef) == true ? (Unit(value(rig), false, penalty(rig)+penalty(lef)), pot) : (Unit(value(lef), false, penalty(lef)+penalty(rig)), pot)

            elseif potential
                return Unit(quantities[end], false, 0), true #returns dummy Unit and true to find all nodes which could potentially be corrected

            elseif correct  # no joker in left or right, unit will be corrected through constant
                if rand(Bool) # correct unit of left, takes on unit of right
                    new_times = Node(2, findfirst(isequal(*), ops.binops))
                    add_unit!(new_times, "[], $(penalty(lef)+1)")
                    new_times.lef = Node(rand())
                    add_unit!(new_times.lef, "[], $(0)")
                    new_times.rig = node.lef
                    node.lef = new_times
                    add_unit!(node, "[$(dimension(rig))], $(penalty(lef)+penalty(rig)+1)")
                    return Unit(value(rig), false, penalty(rig)+penalty(lef)+1), true
                else # correct unit of right, takes on unit of left
                    new_times = Node(2, findfirst(isequal(*), ops.binops))
                    add_unit!(new_times, "[], $(penalty(rig)+1)")
                    new_times.rig = Node(rand())
                    add_unit!(new_times.rig, "[], $(0)")
                    new_times.lef = node.rig
                    node.rig = new_times
                    add_unit!(node, "[], $(penalty(lef)+penalty(rig)+1)")
                    return Unit(value(lef), false, penalty(lef)+penalty(rig)+1), true
                end

            else #unit will not be corrected, only penalty added and the current unit returned (either left or right)
                add_unit!(node, "[$(dimension(lef))], $(penalty(lef)+penalty(rig)+1)")
                return Unit((rand(Bool) == true) ? value(lef) : value(rig), false, penalty(lef)+penalty(rig)+1), true
            end

        elseif op in (:*, :/)
            #check if already using a constant
            if (joker(lef) == true) || (joker(rig) == true)
                add_unit!(node, "[?], $(penalty(lef)+penalty(rig))")
                return Unit(quantities[end], true, penalty(lef)+penalty(rig)), pot
            end
            #return lef op rig
            if op == :*
                add_unit!(node, "[$(dimension(lef)*dimension(rig))], $(penalty(lef)+penalty(rig))")
                return Unit(value(lef) * value(rig), false, penalty(lef)+penalty(rig)), pot
            else
                add_unit!(node, "[$(dimension(lef)/dimension(rig))], $(penalty(lef)+penalty(rig))")
                return Unit(value(lef) / value(rig), false, penalty(lef)+penalty(rig)), pot
            end
        elseif op in (:^,)
            if iszero(dimension(rig))
                add_unit!(node, "[?], $(penalty(rig)+penalty(lef))")
                return Unit(quantities[end], true, penalty(rig)+penalty(lef)), pot
            
            elseif potential
                return Unit(quantities[end], false, 0), true #returns dummy Unit and true to find all nodes which could potentially be corrected

            elseif correct #unit will be corrected through constant
                new_times = Node(2, findfirst(isequal(*), ops.binops))
                add_unit!(new_times, "[], $(penalty(rig)+1)")
                new_times.rig = Node(rand())
                add_unit!(new_times.rig, "[], $(0)")
                new_times.lef = node.rig
                node.rig = new_times
                add_unit!(node, "[], $(penalty(lef)+penalty(rig)+1)")
                return Unit(quantities[end], true, penalty(lef)+penalty(rig)+1), true

            else #unit will not be corrected, only penalty added and the current unit returned
                add_unit!(node, "[$(dimension(lef))], $(penalty(lef)+penalty(rig)+1)")
                return Unit(value(lef), false, penalty(lef)+penalty(rig)+1), true
            end            
        end
    end
end

dim_analysis! (generic function with 1 method)

In [32]:
### this is corrected
struct Unit{Q<:AbstractQuantity}
    value::Q
    joker::Bool
    penalty::Int64
end

ustrip(w::Unit)     = ustrip(w.value)                                   # ustrip removes the unit from the quantity
dimension(w::Unit)  = DynamicQuantities.dimension(w.value)              # dimension returns the dimension of the quantity
penalty(x::Unit)    = x.penalty                                         # penalty returns the penalty of the unit
joker(x::Unit)      = x.joker                                           # joker returns true if the unit is a joker
value(x::Unit)      = x.value                                           # value returns the quantity

function dim_analysis!(node::Node{T}, quantities::Vector{Q}, ops, correct::Bool; potential = false) where {T<:Number, Q<:AbstractQuantity{T}}
    #pot = false
    if node.ari == -1 #constant
        #add_unit!(node, "[], $(0)")
        return Unit{Q}(node.val * quantities[end], true, 0), false

    elseif node.ari == 0 #variable
        #add_unit!(node, "[$(quantities[node.ind])], $(0)")
        return Unit{Q}((node.val * quantities[node.ind]), false, 0), false
        
    elseif node.ari == 1 #unary operation
        lef, pot = dim_analysis!(node.lef, quantities, ops, correct, potential = potential) #lef is of type Unit
        pot && potential && return Unit(quantities[end], false, 0), pot #returns dummy Unit and true to find all nodes which could potentially be corrected
        op = Symbol(ops.unaops[node.ind])

        if op in (:sin, :cos, :tan, :arccos, :arcsin, :exp, :log, :log10)
            #check if lef is dimensionless
            if iszero(dimension(lef))
                #add_unit!(node, "[], $(penalty(lef))")
                return Unit(Quantity(quantities[end]), false, penalty(lef)), pot
            elseif potential
                return Unit(quantities[end], false, 0), true #returns dummy Unit and true to find all nodes which could potentially be corrected
            elseif correct #unit will be corrected through constant
                # correct unit of lef using a constant
                new_times = Node(2, findfirst(isequal(*), ops.binops))
                #add_unit!(new_times, "[], $(penalty(lef)+1)")
                new_times.lef = Node(rand())
                #add_unit!(new_times.lef, "[], $(penalty(lef))")
                new_times.rig = node.lef
                node.lef = new_times
                #add_unit!(node, "[], $(penalty(lef)+1)")
                return Unit(quantities[end], false, 0), true
            else #unit will not be corrected, only penalty added and the current unit returned
                #add_unit!(node, "[$(dimension(lef))], $(penalty(lef)+1)")
                return Unit(value(lef), false, penalty(lef) + 1), true
            end
        elseif op in (:sqrt, :sqrt_abs)
            #add_unit!(node, "[$(DynamicQuantities.dimension(sqrt(value(lef))))], $(penalty(lef))")
            return Unit(sqrt(value(lef)), joker(lef), penalty(lef)), pot
        elseif op in (:abs,)
            #add_unit!(node, "[$(DynamicQuantities.dimension(value(lef)))], $(penalty(lef))")
            return Unit(abs(value(lef)), joker(lef), penalty(lef)), pot
        elseif op in (:pow2,)
            #add_unit!(node, "[$(DynamicQuantities.dimension(value(lef))^2)], $(penalty(lef))")
            return Unit(value(lef)^2, joker(lef), penalty(lef)), pot
        elseif op in (:pow3,)
            #add_unit!(node, "[$(DynamicQuantities.dimension(value(lef))^3)], $(penalty(lef))")
            return Unit(value(lef)^3, joker(lef), penalty(lef)), pot
        end

    elseif node.ari == 2
        lef, pot = dim_analysis!(node.lef, quantities, ops, correct, potential = potential)
        pot && potential && return Unit(quantities[end], false, 0), true #returns dummy Unit and true to find all nodes which could potentially be corrected
        rig, pot = dim_analysis!(node.rig, quantities, ops, correct, potential = potential)
        pot && potential && return Unit(quantities[end], false, 0), true #returns dummy Unit and true to find all nodes which could potentially be corrected
        
        op = Symbol(ops.binops[node.ind])
        if op in (:+, :-)
            
            if true in (joker(lef), joker(rig)) # no correction if either one is already a joker (would add unnecessary constants)
                #add_unit!(node, joker(lef) == true ? "[$(dimension(rig))], $(penalty(lef)+penalty(rig))" : "[$(dimension(lef))], $(penalty(lef)+penalty(rig))")
                # are both jokers?
                if joker(lef) && joker(rig)
                    #add_unit!(node, "[?], $(penalty(lef)+penalty(rig))")
                    return Unit(quantities[end], true, penalty(lef)+penalty(rig)), pot
                end
                return joker(lef) == true ? (Unit(value(rig), false, penalty(rig)+penalty(lef)), pot) : (Unit(value(lef), false, penalty(lef)+penalty(rig)), pot)
            
            elseif dimension(lef) == dimension(rig)
                    #add_unit!(node, "[$(dimension(lef))], $(penalty(lef)+penalty(rig))")
                    return Unit(value(lef), false, penalty(lef)+penalty(rig)), pot
    
            elseif potential
                return Unit(quantities[end], false, 0), true #returns dummy Unit and true to find all nodes which could potentially be corrected

            elseif correct  # no joker in left or right, unit will be corrected through constant
                if rand(Bool) # correct unit of left, takes on unit of right
                    new_times = Node(2, findfirst(isequal(*), ops.binops))
                    #add_unit!(new_times, "[], $(penalty(lef)+1)")
                    new_times.lef = Node(rand())
                    #add_unit!(new_times.lef, "[], $(0)")
                    new_times.rig = node.lef
                    node.lef = new_times
                    #add_unit!(node, "[$(dimension(rig))], $(penalty(lef)+penalty(rig)+1)")
                    return Unit(value(rig), false, 0), true #penalty(lef)+penalty(rig)+1
                else # correct unit of right, takes on unit of left
                    new_times = Node(2, findfirst(isequal(*), ops.binops))
                    #add_unit!(new_times, "[], $(penalty(rig)+1)")
                    new_times.rig = Node(rand())
                    #add_unit!(new_times.rig, "[], $(0)")
                    new_times.lef = node.rig
                    node.rig = new_times
                    #add_unit!(node, "[], $(penalty(lef)+penalty(rig)+1)")
                    return Unit(value(lef), false, 0), true #penalty(lef)+penalty(rig)+1
                end

            else #unit will not be corrected, only penalty added and the current unit returned (either left or right)
                #add_unit!(node, "[$(dimension(lef))], $(penalty(lef)+penalty(rig)+1)")
                return Unit((rand(Bool) == true) ? value(lef) : value(rig), false, penalty(lef)+penalty(rig)+1), true
            end

        elseif op in (:*, :/)
            #check if already using a constant
            if (joker(lef) == true) || (joker(rig) == true)
                #add_unit!(node, "[], $(penalty(lef)+penalty(rig))")
                return Unit(quantities[end], true, penalty(lef)+penalty(rig)), pot
            end
            #return lef op rig
            if op == :*
                #add_unit!(node, "[$(dimension(lef)*dimension(rig))], $(penalty(lef)+penalty(rig))")
                return Unit(value(lef) * value(rig), false, penalty(lef)+penalty(rig)), pot
            else
                #add_unit!(node, "[$(dimension(lef)/dimension(rig))], $(penalty(lef)+penalty(rig))")
                return Unit(value(lef) / value(rig), false, penalty(lef)+penalty(rig)), pot
            end
        elseif op in (:^,)
            if (iszero(dimension(rig)) || joker(rig)) && (iszero(dimension(lef)) || joker(lef))
                #add_unit!(node, "[], $(penalty(rig)+penalty(lef))")
                return Unit(quantities[end], true, penalty(rig)+penalty(lef)), pot
            
            elseif potential
                return Unit(quantities[end], false, 0), true #returns dummy Unit and true to find all nodes which could potentially be corrected

            elseif correct #unit will be corrected through constant
                if (iszero(dimension(rig)) || joker(rig)) # left must be corrected
                    new_times = Node(2, findfirst(isequal(*), ops.binops))
                    #add_unit!(new_times, "[], $(penalty(lef)+1)")
                    new_times.lef = Node(rand())
                    #add_unit!(new_times.lef, "[], $(0)")
                    new_times.rig = node.lef
                    node.lef = new_times
                    #add_unit!(node, "[$(dimension(rig))], $(penalty(lef)+penalty(rig)+1)")
                    return Unit(value(rig), false, 0), true #penalty(lef)+penalty(rig)+1
                else # right must be corrected
                    new_times = Node(2, findfirst(isequal(*), ops.binops))
                    #add_unit!(new_times, "[], $(penalty(rig)+1)")
                    new_times.lef = Node(rand())
                    #add_unit!(new_times.rig, "[], $(0)")
                    new_times.rig = node.rig
                    node.rig = new_times
                    #add_unit!(node, "[], $(penalty(lef)+penalty(rig)+1)")
                end
                return Unit(quantities[end], true, 0), true #penalty(lef)+penalty(rig)+1

            else #unit will not be corrected, only penalty added and the current unit returned
                #add_unit!(node, "[$(dimension(lef))], $(penalty(lef)+penalty(rig)+1)")
                return Unit(value(lef), false, penalty(lef)+penalty(rig)+1), true
            end
        elseif op in (:arctan2,)
            #add_unit!(node, "[$(DynamicQuantities.dimension(atan2(value(lef), value(rig)))], $(penalty(lef)+penalty(rig))")
            if dimension(rig) == dimension(lef)
                return Unit(quantities[end], false, penalty(lef)+penalty(rig)), pot
            else
                return Unit(quantities[end], false, penalty(lef)+penalty(rig)+1), pot
            end
        end
    end
end

┌ Error: Failed to revise /Users/juliareu/GitCode/TiSR/src/dim_analysis.jl
│   exception = (LoadError("/Users/juliareu/GitCode/TiSR/src/dim_analysis.jl", 172, "incomplete: \"function\" at /Users/juliareu/GitCode/TiSR/src/dim_analysis.jl:15 requires end"), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x0000000155bbe263])
└ @ Revise /Users/juliareu/.julia/packages/Revise/snrGv/src/packagedef.jl:725
┌ Warning: The running code does not match the saved version for the following files:
│ 
│   /Users/juliareu/GitCode/TiSR/src/dim_analysis.jl
│ 
│ If the error was due to evaluation order, it can sometimes be resolved by calling `Revise.retry()`.
│ Use Revise.errors() to report errors again. Only the first error in each file is shown.
│ Your prompt color may be yellow until the errors are resolved.
└ @ Revise /Users/juliareu/.julia/packages/Revise/snrGv/src/packagedef.jl:833


dim_analysis! (generic function with 1 method)

In [22]:
### this is old
struct Unit{Q<:AbstractQuantity}
    value::Q
    joker::Bool
    penalty::Int64
end

ustrip(w::Unit)     = ustrip(w.value)                                   # ustrip removes the unit from the quantity
dimension(w::Unit)  = DynamicQuantities.dimension(w.value)              # dimension returns the dimension of the quantity
penalty(x::Unit)    = x.penalty                                         # penalty returns the penalty of the unit
joker(x::Unit)      = x.joker                                           # joker returns true if the unit is a joker
value(x::Unit)      = x.value                                           # value returns the quantity

function dim_analysis!(node::Node{T}, quantities::Vector{Q}, ops, correct::Bool; potential = false) where {T<:Number, Q<:AbstractQuantity{T}}
    #pot = false
    if node.ari == -1 #constant
        #add_unit!(node, "[], $(0)")
        return Unit{Q}(node.val * quantities[end], true, 0), false

    elseif node.ari == 0 #variable
        #add_unit!(node, "[$(quantities[node.ind])], $(0)")
        return Unit{Q}((node.val * quantities[node.ind]), false, 0), false
        
    elseif node.ari == 1 #unary operation
        lef, pot = dim_analysis!(node.lef, quantities, ops, correct, potential = potential) #lef is of type Unit
        pot && potential && return Unit(quantities[end], false, 0), pot #returns dummy Unit and true to find all nodes which could potentially be corrected
        op = Symbol(ops.unaops[node.ind])

        if op in (:sin, :cos, :tan, :arccos, :arcsin, :exp, :log, :log10)
            #check if lef is dimensionless
            if iszero(dimension(lef))
                #add_unit!(node, "[], $(penalty(lef))")
                return Unit(Quantity(quantities[end]), false, penalty(lef)), pot
            elseif potential
                return Unit(quantities[end], false, 0), true #returns dummy Unit and true to find all nodes which could potentially be corrected
            elseif correct #unit will be corrected through constant
                # correct unit of lef using a constant
                new_times = Node(2, findfirst(isequal(*), ops.binops))
                #add_unit!(new_times, "[], $(penalty(lef)+1)")
                new_times.lef = Node(rand())
                #add_unit!(new_times.lef, "[], $(penalty(lef))")
                new_times.rig = node.lef
                node.lef = new_times
                #add_unit!(node, "[], $(penalty(lef)+1)")
                return Unit(quantities[end], false, 0), true
            else #unit will not be corrected, only penalty added and the current unit returned
                #add_unit!(node, "[$(dimension(lef))], $(penalty(lef)+1)")
                return Unit(value(lef), false, penalty(lef) + 1), true
            end
        elseif op in (:sqrt, :sqrt_abs)
            #add_unit!(node, "[$(DynamicQuantities.dimension(sqrt(value(lef))))], $(penalty(lef))")
            return Unit(sqrt(value(lef)), joker(lef), penalty(lef)), pot
        elseif op in (:abs,)
            #add_unit!(node, "[$(DynamicQuantities.dimension(value(lef)))], $(penalty(lef))")
            return Unit(abs(value(lef)), joker(lef), penalty(lef)), pot
        elseif op in (:pow2,)
            #add_unit!(node, "[$(DynamicQuantities.dimension(value(lef))^2)], $(penalty(lef))")
            return Unit(value(lef)^2, joker(lef), penalty(lef)), pot
        elseif op in (:pow3,)
            #add_unit!(node, "[$(DynamicQuantities.dimension(value(lef))^3)], $(penalty(lef))")
            return Unit(value(lef)^3, joker(lef), penalty(lef)), pot
        end

    elseif node.ari == 2
        lef, pot = dim_analysis!(node.lef, quantities, ops, correct, potential = potential)
        pot && potential && return Unit(quantities[end], false, 0), true #returns dummy Unit and true to find all nodes which could potentially be corrected
        rig, pot = dim_analysis!(node.rig, quantities, ops, correct, potential = potential)
        pot && potential && return Unit(quantities[end], false, 0), true #returns dummy Unit and true to find all nodes which could potentially be corrected
        
        op = Symbol(ops.binops[node.ind])
        if op in (:+, :-)
            if dimension(lef) == dimension(rig)
                #add_unit!(node, "[$(dimension(lef))], $(penalty(lef)+penalty(rig))")
                return Unit(value(lef), joker(lef), penalty(lef)+penalty(rig)), pot

            elseif true in (joker(lef), joker(rig)) # no correction if either one is already a joker (would add unnecessary constants)
                #add_unit!(node, joker(lef) == true ? "[$(dimension(rig))], $(penalty(lef)+penalty(rig))" : "[$(dimension(lef))], $(penalty(lef)+penalty(rig))")
                return joker(lef) == true ? (Unit(value(rig), false, penalty(rig)+penalty(lef)), pot) : (Unit(value(lef), false, penalty(lef)+penalty(rig)), pot)

            elseif potential
                return Unit(quantities[end], false, 0), true #returns dummy Unit and true to find all nodes which could potentially be corrected

            elseif correct  # no joker in left or right, unit will be corrected through constant
                if rand(Bool) # correct unit of left, takes on unit of right
                    new_times = Node(2, findfirst(isequal(*), ops.binops))
                    #add_unit!(new_times, "[], $(penalty(lef)+1)")
                    new_times.lef = Node(rand())
                    #add_unit!(new_times.lef, "[], $(0)")
                    new_times.rig = node.lef
                    node.lef = new_times
                    #add_unit!(node, "[$(dimension(rig))], $(penalty(lef)+penalty(rig)+1)")
                    return Unit(value(rig), false, 0), true #penalty(lef)+penalty(rig)+1
                else # correct unit of right, takes on unit of left
                    new_times = Node(2, findfirst(isequal(*), ops.binops))
                    #add_unit!(new_times, "[], $(penalty(rig)+1)")
                    new_times.rig = Node(rand())
                    #add_unit!(new_times.rig, "[], $(0)")
                    new_times.lef = node.rig
                    node.rig = new_times
                    #add_unit!(node, "[], $(penalty(lef)+penalty(rig)+1)")
                    return Unit(value(lef), false, 0), true #penalty(lef)+penalty(rig)+1
                end

            else #unit will not be corrected, only penalty added and the current unit returned (either left or right)
                #add_unit!(node, "[$(dimension(lef))], $(penalty(lef)+penalty(rig)+1)")
                return Unit((rand(Bool) == true) ? value(lef) : value(rig), false, penalty(lef)+penalty(rig)+1), true
            end

        elseif op in (:*, :/)
            #check if already using a constant
            if (joker(lef) == true) || (joker(rig) == true)
                #add_unit!(node, "[], $(penalty(lef)+penalty(rig))")
                return Unit(quantities[end], true, penalty(lef)+penalty(rig)), pot
            end
            #return lef op rig
            if op == :*
                #add_unit!(node, "[$(dimension(lef)*dimension(rig))], $(penalty(lef)+penalty(rig))")
                return Unit(value(lef) * value(rig), false, penalty(lef)+penalty(rig)), pot
            else
                #add_unit!(node, "[$(dimension(lef)/dimension(rig))], $(penalty(lef)+penalty(rig))")
                return Unit(value(lef) / value(rig), false, penalty(lef)+penalty(rig)), pot
            end
        elseif op in (:^,)
            if (iszero(dimension(rig)) || joker(rig)) && (iszero(dimension(lef)) || joker(lef))
                #add_unit!(node, "[], $(penalty(rig)+penalty(lef))")
                return Unit(quantities[end], true, penalty(rig)+penalty(lef)), pot
            
            elseif potential
                return Unit(quantities[end], false, 0), true #returns dummy Unit and true to find all nodes which could potentially be corrected

            elseif correct #unit will be corrected through constant
                if (iszero(dimension(rig)) || joker(rig)) # left must be corrected
                    new_times = Node(2, findfirst(isequal(*), ops.binops))
                    #add_unit!(new_times, "[], $(penalty(lef)+1)")
                    new_times.lef = Node(rand())
                    #add_unit!(new_times.lef, "[], $(0)")
                    new_times.rig = node.lef
                    node.lef = new_times
                    #add_unit!(node, "[$(dimension(rig))], $(penalty(lef)+penalty(rig)+1)")
                    return Unit(value(rig), false, 0), true #penalty(lef)+penalty(rig)+1
                else # right must be corrected
                    new_times = Node(2, findfirst(isequal(*), ops.binops))
                    #add_unit!(new_times, "[], $(penalty(rig)+1)")
                    new_times.lef = Node(rand())
                    #add_unit!(new_times.rig, "[], $(0)")
                    new_times.rig = node.rig
                    node.rig = new_times
                    #add_unit!(node, "[], $(penalty(lef)+penalty(rig)+1)")
                end
                return Unit(quantities[end], true, 0), true #penalty(lef)+penalty(rig)+1

            else #unit will not be corrected, only penalty added and the current unit returned
                #add_unit!(node, "[$(dimension(lef))], $(penalty(lef)+penalty(rig)+1)")
                return Unit(value(lef), false, penalty(lef)+penalty(rig)+1), true
            end
        elseif op in (:arctan2,)
            #add_unit!(node, "[$(DynamicQuantities.dimension(atan2(value(lef), value(rig)))], $(penalty(lef)+penalty(rig))")
            if dimension(rig) == dimension(lef)
                return Unit(quantities[end], false, penalty(lef)+penalty(rig)), pot
            else
                return Unit(quantities[end], false, penalty(lef)+penalty(rig)+1), pot
            end
        end
    end
end

dim_analysis! (generic function with 1 method)

In [4]:
function make_tree(node::Node)
    g = SimpleDiGraph()
    labels = Vector{String}()
    
    function dfs(node, parent)
        if !isnothing(node)
            add_vertex!(g)
            node_id = nv(g)
            if !isnothing(parent)
                add_edge!(g, parent, nv(g))
            end
            if node.ari==0
                push!(labels, "v$(node.ind) $(node.unit)")
            elseif node.ari==-1
                push!(labels, "$(round(node.val, digits=3)) $(node.unit)")
            elseif node.ari==2
                push!(labels, "$(Symbol(ops.binops[node.ind])) $(node.unit)")
                dfs(node.lef, node_id)
                dfs(node.rig, node_id)
            elseif node.ari==1
                push!(labels, "$(Symbol(ops.unaops[node.ind])) $(node.unit)")
                dfs(node.lef, node_id)                
            end            
        end
    end
    dfs(node, nothing)
    return g, labels 
end

function plot_tree(g::SimpleDiGraph, labels)
    xs, ys, paths = solve_positions(Zarate(), g);
    #create color with rgb values 0.5, 0.25, 0.3
    color = RGB(136/255, 148/255, 254/255)

    draw(PDF("tree_without_viol.pdf", 12cm, 12cm), gplot(g, ys, xs, NODESIZE = 0.2, nodelabel=(labels), NODELABELSIZE=3.2, nodefillc = color))
end

plot_tree (generic function with 1 method)

In [14]:
# generate initial random children # -----------------------------------------------------------
start_pop= TiSR.Node[]

while length(start_pop)  < 100
    push!(start_pop, TiSR.grow_equation(ops.general.init_tree_depth, ops))
end

#map(x -> println(x), start_pop);



4-element Vector{Quantity{Float64, Dimensions{DynamicQuantities.FixedRational{Int32, 25200}}}}:
 1.0 K
 1.0 s
 1.0 m
 0.0 

In [33]:
units = [u"m",  u"0", u"0", u"0", u"0", u"0"]

eq = string_to_node("((0.573367851517656-sin((v2+(((-0.739131004145644+v2)/v1)+12.2348916660722))))*(0.0315119515423098-(0.295811502230742/v1)))", ops)
TiSR.reorder_add_n_mul!(eq, ops)
TiSR.simplify_binary_of_param!(eq)
TiSR.simplify_unary_of_param!(eq)

In [34]:
unit, pot = dim_analysis!(eq, units, ops, false)
println(penalty(unit))
println(pot)


2
false


In [71]:
i=1
for ind in start_pop
    println("Index: ", i)
    i = i+1
    println("Original tree:    ", ind)
    TiSR.reorder_add_n_mul!(ind, ops)
    TiSR.simplify_binary_of_param!(ind)
    TiSR.simplify_unary_of_param!(ind)
    unit, pot = dim_analysis!(ind, quantities, ops, false)
    println("Corrected tree:   ", ind)
    println("Penalty: ", penalty(unit))
    println("----------------------")
end

Index: 1
Original tree:    sin(log(pow3(sqrt_abs((0.451*v3)))))

Corrected tree:   sin(log(pow3(sqrt_abs((0.451*v3)))))

Penalty: 0
----------------------
Index: 2
Original tree:    sqrt_abs((sin(((0.544/v2)-(v1/v1)))^0.428))

Corrected tree:   sqrt_abs((sin(((0.544/v2)-(v1/v1)))^0.428))

Penalty: 0
----------------------
Index: 3
Original tree:    (pow2(cos((pow2(v3)/(0.217+v2))))/sqrt_abs(((pow3(v2)/pow2(v2))+(log(v3)/sin(v4)))))

Corrected tree:   (pow2(cos((pow2(v3)/(0.217+v2))))/sqrt_abs(((pow3(v2)/pow2(v2))+(log(v3)/sin(v4)))))

Penalty: 3
----------------------
Index: 4
Original tree:    sqrt_abs(sin(pow2(pow2((0.714*v3)))))

Corrected tree:   sqrt_abs(sin(pow2(pow2((0.714*v3)))))

Penalty: 0
----------------------
Index: 5
Original tree:    ((sin((log(v1)+pow2(v2)))/exp(abs((v1-0.784))))/exp((pow3((0.398+v3))-((v2^0.216)+(0.927/v4)))))

Corrected tree:   ((sin((log(v1)+pow2(v2)))/exp(abs((v1-0.784))))/exp((pow3((0.398+v3))-((v2^0.216)+(0.927/v4)))))

Penalty: 4
----------------

In [72]:
ind = start_pop[10]
print(ind)
g, labels = make_tree(ind)
plot_tree(g, labels)

exp((abs(sin((0.871*v1)))+(pow2((0.495/v1))/pow3((v1^0.206)))))
